In [2]:
import pandas as pd
import numpy as np

In [3]:
data=pd.read_csv('Bone_Tumor_Dataset_Expanded_200k.csv')

In [4]:
data.head

<bound method NDFrame.head of         Patient ID     Sex  Age         Grade  \
0       STS_000001  Female   64  Intermediate   
1       STS_000002    Male   52          High   
2       STS_000003    Male   27  Intermediate   
3       STS_000004    Male   56          High   
4       STS_000005  Female   62          High   
...            ...     ...  ...           ...   
199995  STS_199996  Female   30          High   
199996  STS_199997  Female   43  Intermediate   
199997  STS_199998    Male   56          High   
199998  STS_199999  Female   41  Intermediate   
199999  STS_200000    Male   70          High   

                             Histological type        MSKCC type  \
0                  pleiomorphic leiomyosarcoma               MFH   
1                  pleiomorphic leiomyosarcoma  Synovial sarcoma   
2                  pleiomorphic leiomyosarcoma    Leiomyosarcoma   
3             malignant solitary fibrous tumor    Leiomyosarcoma   
4          sclerosing epithelioid fibrosa

In [5]:
data.columns


Index(['Patient ID', 'Sex', 'Age', 'Grade', 'Histological type', 'MSKCC type',
       'Site of primary STS', 'Status (NED, AWD, D)', 'Treatment'],
      dtype='object')

In [8]:
data['Grade'].unique()

array(['Intermediate', 'High'], dtype=object)

In [9]:
data['Histological type'].unique()

array(['pleiomorphic leiomyosarcoma', 'malignant solitary fibrous tumor',
       'sclerosing epithelioid fibrosarcoma', 'myxoid fibrosarcoma',
       'undifferentiated - pleiomorphic', 'synovial sarcoma',
       'undifferentiated pleomorphic liposarcoma', 'epithelioid sarcoma',
       'poorly differentiated synovial sarcoma',
       'pleiomorphic spindle cell undifferentiated',
       'pleomorphic sarcoma', 'myxofibrosarcoma', 'leiomyosarcoma'],
      dtype=object)

In [10]:
data['MSKCC type'].unique()

array(['MFH', 'Synovial sarcoma', 'Leiomyosarcoma'], dtype=object)

In [11]:
data['Site of primary STS'].unique()

array(['left thigh', 'right thigh', 'right parascapusular', 'left biceps',
       'right buttock', 'parascapusular', 'left buttock'], dtype=object)

In [ ]:
data['Status (NED, AWD, D)'].unique()

array(['NED', 'D', 'AWD'], dtype=object)

In [14]:
data['Treatment'].unique()

array(['Radiotherapy + Surgery', 'Radiotherapy + Surgery + Chemotherapy',
       'Surgery + Chemotherapy'], dtype=object)

In [15]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import confusion_matrix, classification_report
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Conv1D, GlobalMaxPooling1D, LSTM, Reshape

ModuleNotFoundError: No module named 'tensorflow'